XOR을 NAND와 OR, AND Gate의 조합으로 구현

In [8]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def numerical_derivative(f, x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]

        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x)

        x[idx] = tmp_val - delta_x
        fx2 = f(x)

        grad[idx] = (fx1 - fx2) / (2*delta_x)
        x[idx] = tmp_val
        it.iternext()
    
    return grad

class LogicGate:

    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name

        self.__xdata = xdata.reshape(4, 2)
        self.__tdata = tdata.reshape(4, 1)
        self.__W = np.random.rand(2, 1)
        self.__b = np.random.rand(1)

        self.__learning_rate = 1e-2
    
    def loss_func(self):
        delta = 1e-7

        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)

        return -np.sum( self.__tdata*np.log(y+delta) + (1-self.__tdata)*np.log((1-y)+delta) )

    def error_val(self):
        delta = 1e-7

        z = np.dot(self.__xdata, self.__W) + self.__b
        y = sigmoid(z)

        return -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log( (1-y)+delta ) )

    def train(self):
        f = lambda x : self.loss_func()
        
        for step in range(8001):
            self.__W -= self.__learning_rate * numerical_derivative(f, self.__W)
            self.__b -= self.__learning_rate * numerical_derivative(f, self.__b)


    def predict(self, input_data):

        z = np.dot(input_data, self.__W) + self.__b
        y = sigmoid(z)

        if y > 0.5:
            result = 1
        else:
            result = 0
        return y, result


In [9]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1]])
AND_tdata = np.array([0, 0, 0, 1])
OR_tdata = np.array([0, 1, 1, 1])
NAND_tdata = np.array([1, 1, 1, 0])

input_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1]])

s1 = []  # NAND 출력
s2 = []  # OR 출력

new_input_data = []     # AND 입력
final_output_data = []  # AND 출력 = XOR 수행 결과

NAND_obj = LogicGate("NAND_GATE", xdata, NAND_tdata)
OR_obj = LogicGate("OR_GATE", xdata, OR_tdata)
AND_obj = LogicGate("AND_GATE", xdata, AND_tdata)

NAND_obj.train()
OR_obj.train()
AND_obj.train()

for index in range(len(input_data)):
    s1 = NAND_obj.predict(input_data[index])
    s2 = OR_obj.predict(input_data[index])

    new_input_data.append(s1[-1])
    new_input_data.append(s2[-1])

    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))

    final_output_data.append(logical_val)
    new_input_data = []

In [10]:
for index in range(len(input_data)):
    print(input_data[index], " = ", final_output_data[index], end='')
    print("\n")

[0 0]  =  0

[0 1]  =  1

[1 0]  =  1

[1 1]  =  0



- XOR 문제는 Multi-Layer로 해결할 수 있음
- 각 gate는 Logistic Regression(Classification) 시스템으로 구성된다.
- 신경망 기반의 딥러닝 핵심 아이디어!

# Logistic Regression을 다양하게 조합하여 미래 값을 예측하는 딥러닝